In [ ]:
import osmnx as ox
import networkx as nx
import pickle 
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import chisquare
import matplotlib.patches as mpatches
from scipy.sparse import spdiags
from scipy.linalg import eigh
from sklearn.cluster import KMeans
from scipy.stats import zscore
import matplotlib.pyplot as plt
import multiprocessing

plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams['font.size'] = 14

path = 'C:/Users/Besitzer/Desktop/Geospatial Analysis/Data'
os.chdir('C:/Users/Besitzer/Desktop/Geospatial Analysisb

In [ ]:
city_list = []
for file in os.listdir(path):
    city = file.replace('.pkl', '')
    city_list.append(city)
    
def cluster(b):
    dist_citys = []
    distance_progress = [] 

    #if file == 'Amsterdam.pkl':
    #print(file.replace('.pkl',''))
    data = pd.read_pickle(path + '/' + city_list[b] + '.pkl')

    G = pickle.load( open( 'SN/' + 'network_graph_POI_' + city_list[b] + ".p", "rb" ) )
#     for u, v, key, data in Gu.edges(keys=True, data=True):
#             if data['oneway']:
#                 data['oneway'] = False
    c_node = data['center'][0]
    distance = []
    cum_distance = 0

    dist_list = []
    dist = []

    i = 0
    for node in G.nodes():
        try:
            di = nx.shortest_path_length(G, source=node, target=c_node, weight='length')
            cum_distance += di
            dist.append(di)
        except:
            i += 1
            dist.append(np.inf)
    dist_list.append(dist)
    mean_distance = cum_distance / (len(G.nodes())-i)
    distance.append(mean_distance)
    #print('Mean distance:', mean_distance) #3626.2361939018565

    nodes = ox.graph_to_gdfs(G, edges=False)

    ## Cluster
    #Amount of clusters: c = 2,7 
    for c in range(2,7):
        dist = []
        X = pd.concat([nodes['x'], nodes['y']], axis=1)
        kmeans = KMeans(n_clusters=c, random_state=0).fit(X)
        groupings = kmeans.labels_
        centers = kmeans.cluster_centers_
        clust_dist = []
        a = pd.DataFrame(columns=range(c), index=G.nodes())
        a['category'] = np.nan
        l = list(range(c))
        tn = []
        nns = []

        #Create shortest path to each center
        for i in range(c):  
            destination = (centers[i][1], centers[i][0])
            target_node = ox.get_nearest_node(G, destination)
            tn.append(target_node)            
            for node in G.nodes():
                try:
                    length = nx.shortest_path_length(G, source=node, target=target_node, weight='length')
                    a[i][node] = length
                except:
                    a[i][node] = np.inf

        #Select Cluster
        for i in range(len(a)):
            a['category'].iloc[i] = min(range(len(a.iloc[i])-1), key=a.iloc[i].__getitem__)     

        #for each cluster...
        wrong_nodes = []
        for i in range(c):
            cl = []
            cl = [ind for ind in a.index if a.loc[ind]['category'] == i]
            #Coordinates
            y1 = [G.nodes()[n]['y'] for n in cl]
            x1 = [G.nodes()[n]['x'] for n in cl]
            #Z-Values
            z = np.abs(zscore(list(zip(y1,x1))))
            #Select the most central nodes 
            nodes1 = [cl[ele] for ele in np.where(z < 0.5)[0]]
            #nodes to select mean node and try to calculate the shortest path in the subplot
            #if node path does not exist --> remove this node from cluster cl
            c_new = [j for j in cl if j in nodes1]
            y2 = [G.nodes()[n]['y'] for n in c_new]
            x2 = [G.nodes()[n]['x'] for n in c_new]
            #Center
            center_y1 = sum(y2)/len(y2)
            center_x1 = sum(x2)/len(x2)
            #center node in cluster
            c_node = ox.get_nearest_node(G, (center_y1, center_x1))
            #Cluster as subplot nodes are cl
            #nodes to remove from graph
            rem_nodes = [node for node in G.nodes() if node not in cl]

            H = G.copy()
            nodes2 = [cl[ele] for ele in np.where(z > 2.5)[0]]
            for node in nodes2:
                try: H.remove_node(node)
                except: i += 0 #print(node, 'is already away1')
            for node in rem_nodes:
                try: H.remove_node(node)
                except: i += 0 #print(node, 'is already away2')
            clu = cl
            for node in clu:
                try:
                    length = nx.shortest_path_length(H, source=node, target=c_node, weight='length')
                except:
                    a['category'][node] = np.nan
                    cl.remove(node)
                    wrong_nodes.append(node)
        #nur die verbundenen sind übrig
        #schauen, wo die falschen dazugehören
        for _ in range(25):
            for node in wrong_nodes:
                #if there are neighbors
                if len([n for n in G.neighbors(node)]) != 0:
                    #look if a neighbor has a category
                    for i in [n for n in G.neighbors(node)]:
                        if a['category'][i] is not np.nan:
                            a['category'][node] = a['category'][i]  

        for i in range(c):
            cl = []
            cl = [ind for ind in a.index if a.loc[ind]['category'] == i]
            #Coordinates new
            y1 = [G.nodes()[n]['y'] for n in cl]
            x1 = [G.nodes()[n]['x'] for n in cl]
            #New Center
            center_y1 = sum(y1)/len(y1)
            center_x1 = sum(x1)/len(x1)
            #Nearest Node to Center
            nn = ox.get_nearest_node(G, (center_y1, center_x1))
            nns.append(nn)
            #calculate mean distance of this cluster
            for node in cl:
                try:
                    length = nx.shortest_path_length(G, source=node, target=nn, weight='length')
                    #dist.append(length)
                    a[i][node] = length
                except:
                    a[i][node] = np.inf 
                    #dist.append(np.inf)

        #it could be that the mean nodes are plotted with the wrong color
        for i in range(len(nns)):
            a['category'][nns[i]] = i

        #calculate distance 
        j = 0
        k = 0
        for i in range(len(a)):
            if min(a[l].iloc[i]) != np.inf:
                j += min(a[l].iloc[i])
                k += 1
                dist.append(j)
            else:
                print('Infite way')
                dist.append(np.inf)
        dist_list.append(dist)
        mean = j/k
        distance.append(mean)
        #print('Mean distance:', mean, '; ', len(a)-k, '-Values with infinite distance' )
        a['category'].value_counts(dropna=False)
        colors = ['firebrick', 'steelblue', 'olivedrab', 'goldenrod', 'mediumpurple', 'lightseagreen']
        ec = []
        for i in G.nodes():
            try:
                ec.append(colors[int(a.loc[i]['category'])])
            except:
                ec.append('gray')
        #ec = ['r' if a.loc[i]['category'] == 0 else 'g' for i in Gu.nodes()]
        ns = [100 if node in nns else 5 for node in G.nodes()]
        fig, ax = ox.plot_graph(G, node_size=ns, bgcolor='w', node_color=ec, edge_linewidth=0.4, edge_alpha=1, show=False, close=False)
        ax.set_title(city_list[b] + ' in ' + str(c) + ' Cluster', size=14)
        plt.tight_layout()
        plt.savefig('Cluster/' + city_list[b] + '_' +  str(c) + '.pdf')
        plt.show()
    dist_citys.append(dist_list)
    distance_progress.append(distance)
    open_file = open('Distance/distance_pro_' + city_list[b] + '.pkl', "wb")
    pickle.dump(distance_progress, open_file)
    open_file.close()

    open_file = open('Distance/distance_dist_' + city_list[b] + '.pkl', "wb")
    pickle.dump(dist_citys, open_file)
    open_file.close()
        

if __name__ == "__main__":
    jobs = []
    for i in range(58):
        process = multiprocessing.Process(target=cluster(i))
        jobs.append(process)

    # Start the processes (i.e. calculate the random number lists)      
    for j in jobs:
        j.start()

    # Ensure all of the processes have finished
    for j in jobs:
        j.join()

In [ ]:
city_list = []
for file in os.listdir(path):
    if file == 'Amsterdam.pkl':
        #print(file.replace('.pkl',''))
        data = pd.read_pickle(path + '/' + file)
        city = file.replace('.pkl', '')
        city_list.append(city)

        G = pickle.load( open( 'SN/' + 'network_graph_POI_' + city + ".p", "rb" ) )
    #     for u, v, key, data in Gu.edges(keys=True, data=True):
    #             if data['oneway']:
    #                 data['oneway'] = False
        c_node = data['center'][0]
        distance_progress = []
        distance = []
        cum_distance = 0

        dist_citys = []
        dist_list = []
        dist = []

        i = 0
        for node in G.nodes():
            try:
                di = nx.shortest_path_length(G, source=node, target=c_node, weight='length')
                cum_distance += di
                dist.append(di)
            except:
                i += 1
                dist.append(np.inf)
        dist_list.append(dist)
        mean_distance = cum_distance / (len(G.nodes())-i)
        distance.append(mean_distance)
        #print('Mean distance:', mean_distance) #3626.2361939018565

        nodes = ox.graph_to_gdfs(G, edges=False)

        ## Cluster
        #Amount of clusters: c = 2,7 
        for c in range(2,7):
            X = pd.concat([nodes['x'], nodes['y']], axis=1)
            kmeans = KMeans(n_clusters=c, random_state=0).fit(X)
            groupings = kmeans.labels_
            centers = kmeans.cluster_centers_
            clust_dist = []
            a = pd.DataFrame(columns=range(c), index=G.nodes())
            a['category'] = np.nan
            b = a
            l = list(range(c))
            tn = []
            nns = []

            #Create shortest path to each center
            for i in range(c):  
                destination = (centers[i][1], centers[i][0])
                target_node = ox.get_nearest_node(G, destination)
                tn.append(target_node)            
                for node in G.nodes():
                    try:
                        length = nx.shortest_path_length(G, source=node, target=target_node, weight='length')
                        a[i][node] = length
                    except:
                        a[i][node] = np.inf

            #Select Cluster
            for i in range(len(a)):
                a['category'].iloc[i] = min(range(len(a.iloc[i])-1), key=a.iloc[i].__getitem__)     

            #for each cluster...
            wrong_nodes = []
            for i in range(c):
                cl = []
                cl = [ind for ind in a.index if a.loc[ind]['category'] == i]
                #Coordinates
                y1 = [G.nodes()[n]['y'] for n in cl]
                x1 = [G.nodes()[n]['x'] for n in cl]
                #Z-Values
                z = np.abs(zscore(list(zip(y1,x1))))
                #Select the most central nodes 
                nodes1 = [cl[ele] for ele in np.where(z < 0.5)[0]]
                #nodes to select mean node and try to calculate the shortest path in the subplot
                #if node path does not exist --> remove this node from cluster cl
                c_new = [j for j in cl if j in nodes1]
                y2 = [G.nodes()[n]['y'] for n in c_new]
                x2 = [G.nodes()[n]['x'] for n in c_new]
                #Center
                center_y1 = sum(y2)/len(y2)
                center_x1 = sum(x2)/len(x2)
                #center node in cluster
                c_node = ox.get_nearest_node(G, (center_y1, center_x1))
                #Cluster as subplot nodes are cl
                #nodes to remove from graph
                rem_nodes = [node for node in G.nodes() if node not in cl]

                H = G.copy()
                nodes2 = [cl[ele] for ele in np.where(z > 2)[0]]
                for node in nodes2:
                    try: H.remove_node(node)
                    except: i += 0#print(node, 'is already away1')
                for node in rem_nodes:
                    try: H.remove_node(node)
                    except: i += 0 #print(node, 'is already away2')
                clu = cl
                for node in clu:
                    try:
                        length = nx.shortest_path_length(H, source=node, target=c_node, weight='length')
                    except:
                        a['category'][node] = np.nan
                        cl.remove(node)
                        wrong_nodes.append(node)
            #nur die verbundenen sind übrig
            #schauen, wo die falschen dazugehören
            for _ in range(10):
                for node in wrong_nodes:
                    #if there are neighbors
                    if len([n for n in G.neighbors(node)]) != 0:
                        #look if a neighbor has a category
                        for i in [n for n in G.neighbors(node)]:
                            if a['category'][i] is not np.nan:
                                a['category'][node] = a['category'][i]  

            for i in range(c):
                cl = []
                cl = [ind for ind in a.index if a.loc[ind]['category'] == i]
                #Coordinates new
                y1 = [G.nodes()[n]['y'] for n in cl]
                x1 = [G.nodes()[n]['x'] for n in cl]
                #New Center
                center_y1 = sum(y1)/len(y1)
                center_x1 = sum(x1)/len(x1)
                #Nearest Node to Center
                nn = ox.get_nearest_node(G, (center_y1, center_x1))
                nns.append(nn)
                #calculate mean distance of this cluster
                for node in cl:
                    try:
                        length = nx.shortest_path_length(G, source=node, target=nn, weight='length')
                        clust_dist.append(length)
                        a[i][node] = length
                    except:
                        a[i][node] = np.inf 
            
            #it could be that the mean nodes are plotted with the wrong color
            for i in range(len(nns)):
                a['category'][nns[i]] = i
            
            #calculate distance 
            j = 0
            k = 0
            for i in range(len(a)):
                if min(a[l].iloc[i]) != np.inf:
                    j += min(a[l].iloc[i])
                    k += 1
                    dist.append(min(a[l].iloc[i]))
                else:
                    dist.append(np.inf)
            dist_list.append(dist)
            mean = j/k
            distance.append(mean)
            #print('Mean distance:', mean, '; ', len(a)-k, '-Values with infinite distance' )
            a['category'].value_counts(dropna=False)
            colors = ['firebrick', 'steelblue', 'olivedrab', 'goldenrod', 'mediumpurple', 'lightseagreen']
            ec = []
            for i in G.nodes():
                try:
                    ec.append(colors[int(a.loc[i]['category'])])
                except:
                    ec.append('gray')
            #ec = ['r' if a.loc[i]['category'] == 0 else 'g' for i in Gu.nodes()]
            ns = [100 if node in nns else 5 for node in G.nodes()]
            fig, ax = ox.plot_graph(G, node_size=ns, bgcolor='w', node_color=ec, edge_linewidth=0.4, edge_alpha=1, show=False, close=False)
            ax.set_title(file.replace('.pkl', '') + ' in ' + str(c) + ' Cluster', size=14)
            plt.tight_layout()
            plt.savefig('Cluster/' + city + '_' +  str(c) + '.pdf')
            plt.show()
        dist_citys.append(dist_list)
        distance_progress.append(distance)
    open_file = open('distance_pro_cities.pkl', "wb")
    pickle.dump(distance_progress, open_file)
    open_file.close()

    open_file = open('distance_dist_cities.pkl', "wb")
    pickle.dump(dist_citys, open_file)
    open_file.close()

In [ ]:
import time
import multiprocessing

In [ ]:
data = pd.read_pickle(path + '/' + 'Amsterdam.pkl')
# city = file.replace('.pkl', '')
# city_list.append(city)

G = pickle.load( open( 'C:/Users/Besitzer/Desktop/Geospatial Analysis/SN/' + 'network_graph_POI_' + 'Amsterdam' + ".p", "rb" ) )

c = 2

nodes = ox.graph_to_gdfs(G, edges=False)
X = pd.concat([nodes['x'], nodes['y']], axis=1)
kmeans = KMeans(n_clusters=c, random_state=0).fit(X)
groupings = kmeans.labels_
centers = kmeans.cluster_centers_
clust_dist = []
a = pd.DataFrame(columns=range(c), index=G.nodes())
a['category'] = np.nan
b = a
l = list(range(c))
tn = []
nns = []

#Create shortest path to each center
for i in range(c):  
    destination = (centers[i][1], centers[i][0])
    target_node = ox.get_nearest_node(G, destination)
    tn.append(target_node)            
    for node in G.nodes():
        try:
            length = nx.shortest_path_length(G, source=node, target=target_node, weight='length')
            a[i][node] = length
        except:
            a[i][node] = np.inf
                    
#Select Cluster
for i in range(len(a)):
    a['category'].iloc[i] = min(range(len(a.iloc[i])-1), key=a.iloc[i].__getitem__)      
    
    
    
    
    
    

#for each cluster...
wrong_nodes = []
for i in range(c):
    cl = []
    cl = [ind for ind in a.index if a.loc[ind]['category'] == i]
    #Coordinates
    y1 = [G.nodes()[n]['y'] for n in cl]
    x1 = [G.nodes()[n]['x'] for n in cl]
    #Z-Values
    z = np.abs(zscore(list(zip(y1,x1))))
    #Select the most central nodes 
    nodes1 = [cl[ele] for ele in np.where(z < 0.5)[0]]
    #nodes to select mean node and try to calculate the shortest path in the subplot
    #if node path does not exist --> remove this node from cluster cl
    c_new = [j for j in cl if j in nodes1]
    y2 = [G.nodes()[n]['y'] for n in c_new]
    x2 = [G.nodes()[n]['x'] for n in c_new]
    #Center
    center_y1 = sum(y2)/len(y2)
    center_x1 = sum(x2)/len(x2)
    #center node in cluster
    c_node = ox.get_nearest_node(G, (center_y1, center_x1))
    #Cluster as subplot nodes are cl
    #nodes to remove from graph
    rem_nodes = [node for node in G.nodes() if node not in cl]
    
    H = G.copy()
    nodes2 = [cl[ele] for ele in np.where(z > 3)[0]]
    for node in nodes2:
        H.remove_node(node)
    for node in rem_nodes:
        try:
            H.remove_node(node)
        else:
            #node is already deleted
    clu = cl
    for node in clu:
        try:
            length = nx.shortest_path_length(H, source=node, target=c_node, weight='length')
        except:
            a['category'][node] = np.nan
            cl.remove(node)
            wrong_nodes.append(node)
#nur die verbundenen sind übrig
#schauen, wo die falschen dazugehören
for _ in range(10):
    for node in wrong_nodes:
        #if there are neighbors
        if len([n for n in G.neighbors(node)]) != 0:
            #look if a neighbor has a category
            for i in [n for n in G.neighbors(2054127618)]:
                if a['category'][i] is not np.nan:
                    a['category'][node] = a['category'][i]

In [ ]:
colors = ['firebrick', 'steelblue', 'olivedrab', 'goldenrod', 'mediumpurple', 'lightseagreen']
ec = []
for i in G.nodes():
    try:
        ec.append(colors[int(a.loc[i]['category'])])
    except:
        ec.append('gray')
#ec = ['r' if a.loc[i]['category'] == 0 else 'g' for i in Gu.nodes()]
ns = [100 if node in nns else 5 for node in G.nodes()]
fig, ax = ox.plot_graph(G, node_size=ns, bgcolor='w', node_color=ec, edge_linewidth=0.4, edge_alpha=1, show=False, close=False)
ax.set_title('Amsterdam' + ' in ' + str(c) + ' Cluster', size=14)
plt.tight_layout()
#plt.savefig('Plots/cluster' + str(c) + '_' + city + '.pdf')
plt.show()

In [ ]:
a['category'].value_counts(dropna=False)

In [ ]:
len(H.nodes())

In [ ]:
a['category'].iloc[1] 

In [ ]:
a.loc[node]['category'] = np.nan

In [ ]:
a.loc[node]['category']

In [ ]:
t = np.inf

In [ ]:
t is not np.inf